<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch.

5. you have to decay learning based on below conditions
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%.
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training.

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below.
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process.
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process.
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process.
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [ ]:
import tensorflow as tf
#enabled to get instant output. if you don't need, you can use session concept which was dicussed in lecture videos.
tf.enable_eager_execution()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
Y_train = tf.keras.utils.to_categorical(y_train, 10)
Y_test = tf.keras.utils.to_categorical(y_test, 10)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(60000, 784)
(10000, 784)
(60000, 10)
(10000, 10)


In [ ]:
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

In [ ]:
class Callback(object):

    """Abstract base class used to build new callbacks.
      Attributes:
          params: dict. Training parameters
              (eg. verbosity, batch size, number of epochs...).
          model: instance of `keras.models.Model`.
              Reference of the model being trained.
          validation_data: Deprecated. Do not use.
      The `logs` dictionary that callback methods
      take as argument will contain keys for quantities relevant to
      the current batch or epoch.
      Currently, the `.fit()` method of the `Model` class
      will include the following quantities in the `logs` that
      it passes to its callbacks:
          on_epoch_end: logs include `acc` and `loss`, and
          optionally include `val_loss`
          (if validation is enabled in `fit`), and `val_acc`
          (if validation and accuracy monitoring are enabled).
          on_batch_begin: logs include `size`,
          the number of samples in the current batch.
          on_batch_end: logs include `loss`, and optionally `acc`
            (if accuracy monitoring is enabled).
      """

    def __init__(self):
        self.validation_data = None
        self.model = None
        # Whether this Callback should only run on the chief worker in a
        # Multi-Worker setting.
        # TODO(omalleyt): Make this attr public once solution is stable.
        self._chief_worker_only = None

    def set_params(self, params):
        self.params = params

    def set_model(self, model):
        self.model = model

    def on_batch_begin(self, batch, logs=None):
        """A backwards compatibility alias for `on_train_batch_begin`."""

    def on_batch_end(self, batch, logs=None):
        """A backwards compatibility alias for `on_train_batch_end`."""

    def on_epoch_begin(self, epoch, logs=None):
        """Called at the start of an epoch.
        Subclasses should override for any actions to run. This function should only
        be called during TRAIN mode.
        Arguments:
            epoch: integer, index of epoch.
            logs: dict. Currently no data is passed to this argument for this method
              but that may change in the future.
        """

    def on_epoch_end(self, epoch, logs=None):
        """Called at the end of an epoch.
        Subclasses should override for any actions to run. This function should only
        be called during TRAIN mode.
        Arguments:
            epoch: integer, index of epoch.
            logs: dict, metric results for this training epoch, and for the
              validation epoch if validation is performed. Validation result keys
              are prefixed with `val_`.
        """

    def on_train_batch_begin(self, batch, logs=None):
        """Called at the beginning of a training batch in `fit` methods.
        Subclasses should override for any actions to run.
        Arguments:
            batch: integer, index of batch within the current epoch.
            logs: dict. Has keys `batch` and `size` representing the current batch
              number and the size of the batch.
        """
        # For backwards compatibility.
        self.on_batch_begin(batch, logs=logs)

    def on_train_batch_end(self, batch, logs=None):
        """Called at the end of a training batch in `fit` methods.
        Subclasses should override for any actions to run.
        Arguments:
            batch: integer, index of batch within the current epoch.
            logs: dict. Metric results for this batch.
        """
        # For backwards compatibility.
        self.on_batch_end(batch, logs=logs)

    def on_test_batch_begin(self, batch, logs=None):
        """Called at the beginning of a batch in `evaluate` methods.
        Also called at the beginning of a validation batch in the `fit`
        methods, if validation data is provided.
        Subclasses should override for any actions to run.
        Arguments:
            batch: integer, index of batch within the current epoch.
            logs: dict. Has keys `batch` and `size` representing the current batch
                  number and the size of the batch.
        """

    def on_test_batch_end(self, batch, logs=None):
        """Called at the end of a batch in `evaluate` methods.
        Also called at the end of a validation batch in the `fit`
        methods, if validation data is provided.
        Subclasses should override for any actions to run.
        Arguments:
            batch: integer, index of batch within the current epoch.
            logs: dict. Metric results for this batch.
        """

    def on_predict_batch_begin(self, batch, logs=None):
        """Called at the beginning of a batch in `predict` methods.
        Subclasses should override for any actions to run.
        Arguments:
            batch: integer, index of batch within the current epoch.
            logs: dict. Has keys `batch` and `size` representing the current batch
                  number and the size of the batch.
        """

    def on_predict_batch_end(self, batch, logs=None):
        """Called at the end of a batch in `predict` methods.
        Subclasses should override for any actions to run.
        Arguments:
            batch: integer, index of batch within the current epoch.
            logs: dict. Metric results for this batch.
        """

    def on_train_begin(self, logs=None):
        """Called at the beginning of training.
        Subclasses should override for any actions to run.
        Arguments:
            logs: dict. Currently no data is passed to this argument for this method
                  but that may change in the future.
        """

    def on_train_end(self, logs=None):
        """Called at the end of training.
        Subclasses should override for any actions to run.
        Arguments:
            logs: dict. Currently no data is passed to this argument for this method
                  but that may change in the future.
        """

    def on_test_begin(self, logs=None):
        """Called at the beginning of evaluation or validation.
        Subclasses should override for any actions to run.
        Arguments:
            logs: dict. Currently no data is passed to this argument for this method
              but that may change in the future.
        """

    def on_test_end(self, logs=None):
        """Called at the end of evaluation or validation.
        Subclasses should override for any actions to run.
        Arguments:
            logs: dict. Currently no data is passed to this argument for this method
              but that may change in the future.
        """

    def on_predict_begin(self, logs=None):
        """Called at the beginning of prediction.
        Subclasses should override for any actions to run.
        Arguments:
            logs: dict. Currently no data is passed to this argument for this method
              but that may change in the future.
        """

    def on_predict_end(self, logs=None):
        """Called at the end of prediction.
        Subclasses should override for any actions to run.
        Arguments:
            logs: dict. Currently no data is passed to this argument for this method
              but that may change in the future.
    """

Micro F1 score and AUC score.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
from tensorflow.keras.callbacks import TerminateOnNaN

In [ ]:
from keras.optimizers import SGD

In [ ]:
from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense


model-1

In [ ]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score
import datetime
class Metrics(tf.keras.callbacks.Callback):
  def __init__(self,validation_data=(X_test,Y_test),  interval=1):
        self.interval = interval
        self.X_val, self.y_val = validation_data
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []
    self.losses = []
    self.lr = []
  def on_epoch_end(self, epoch,logs={}):
        y_pred_train = self.model.predict(self.X_val)
        roc_train = roc_auc_score(self.y_val, y_pred_train)
        y_pred_val = self.model.predict(self.X_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        val_predict = (np.asarray(self.model.predict(self.X_val)))
        val_targ = self.y_val
        val_predict=np.argmax(val_predict, axis=1)
        val_targ=np.argmax(val_targ, axis=1)
        _val_f1 = f1_score(val_targ, val_predict, average= 'micro')
        _val_recall = recall_score(val_targ, val_predict, average= 'micro')
        _val_precision = precision_score(val_targ, val_predict, average= 'micro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        self.losses.append(logs.get('loss'))
        self.lr.append(scheduler(len(self.losses)))
        print("\rroc_auc-score: %s - f1-score: %s " %(str(round(roc_train,4)),str(round(roc_val,4))),end=100*' '+'\n')


import math
def scheduler(epoch):
  initial_lrate = 0.1
  drop = 0.05
  epochs_drop = 3.0
  lrate = initial_lrate * math.pow(drop,
  math.floor((1+epoch)/epochs_drop))
  return lrate

m1= keras.models.Sequential()
m1.add(tf.keras.layers.Dense(units =50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1),input_shape=(784,)))
m1.add(tf.keras.layers.Dense(units =50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))
m1.add(tf.keras.layers.Dense(units =50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))
m1.add(tf.keras.layers.Dense(units =50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))
m1.add(tf.keras.layers.Dense(units =50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))
m1.add(tf.keras.layers.Dense(units =10,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))

history_own = Metrics()
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0, patience=1, min_lr=0.1)
terminate = tf.keras.callbacks.TerminateOnNaN()
callback12 = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=2)
filepath="weights-improvement-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',  verbose=1,save_weights_only=False, save_best_only=False , mode='max')
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,write_grads=True)


m1.compile(optimizer=tf.keras.optimizers.SGD(0.1),loss='binary_crossentropy',metrics=['accuracy'])
m1.fit(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test),batch_size=16,callbacks=[history_own,reduce_lr,terminate,callback12,checkpoint,callback,tensorboard_callback ])





Train on 60000 samples, validate on 10000 samples
Epoch 1/5
roc_auc-score: 0.4987 - f1-score: 0.4987                                                                                                     

Epoch 00001: saving model to weights-improvement-01-0.9000.hdf5
60000/60000 [==============================] - 9s 151us/sample - loss: 0.3263 - acc: 0.9000 - val_loss: 0.3252 - val_acc: 0.9000
Epoch 2/5
roc_auc-score: 0.5017 - f1-score: 0.5017                                                                                                     

Epoch 00002: saving model to weights-improvement-02-0.9000.hdf5
60000/60000 [==============================] - 9s 144us/sample - loss: 0.3258 - acc: 0.9000 - val_loss: 0.3259 - val_acc: 0.9000
Epoch 3/5
roc_auc-score: 0.5002 - f1-score: 0.5002                                                                                                     

Epoch 00003: saving model to weights-improvement-03-0.9000.hdf5
60000/60000 [============================

In [ ]:
!rm -rf ./logs/

model-2

In [ ]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score
import datetime
class Metrics(tf.keras.callbacks.Callback):
  def __init__(self,validation_data=(X_test,Y_test),  interval=1):
        self.interval = interval
        self.X_val, self.y_val = validation_data
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []
  def on_epoch_end(self, epoch,logs={}):
        y_pred_train = self.model.predict(self.X_val)
        roc_train = roc_auc_score(self.y_val, y_pred_train)
        y_pred_val = self.model.predict(self.X_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        val_predict = (np.asarray(self.model.predict(self.X_val)))
        val_targ = self.y_val
        val_predict=np.argmax(val_predict, axis=1)
        val_targ=np.argmax(val_targ, axis=1)
        _val_f1 = f1_score(val_targ, val_predict, average= 'micro')
        _val_recall = recall_score(val_targ, val_predict, average= 'micro')
        _val_precision = precision_score(val_targ, val_predict, average= 'micro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("\rroc-auc_train: %s - roc-auc_val: %s " %(str(round(roc_train,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return
def scheduler(epoch,lr):
   if(epoch%3==0):
     lr *= 0.95
   return lr




m2= keras.models.Sequential()
m2.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1),input_shape=(784,)))
m2.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))
m2.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))
m2.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))
m2.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))
m2.add(tf.keras.layers.Dense(units =10,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(0,1)))


history_own = Metrics()
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0, patience=1, min_lr=0.1)
terminate = tf.keras.callbacks.TerminateOnNaN()
callback12 = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=2)
filepath="weights-improvement-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',  verbose=1,save_weights_only=False, save_best_only=False , mode='max')
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,write_grads=True)


m2.compile(optimizer=tf.keras.optimizers.SGD(0.1),loss='binary_crossentropy',metrics=['accuracy'])
m2.fit(X_train,Y_train,epochs=3,validation_data=(X_test,Y_test),batch_size=16,callbacks=[history_own,reduce_lr,terminate,callback12,checkpoint,callback,tensorboard_callback2 ])


Train on 60000 samples, validate on 10000 samples
Epoch 1/3
roc-auc_train: 0.5 - roc-auc_val: 0.5                                                                                                     

Epoch 00001: saving model to weights-improvement-01-0.8202.hdf5
60000/60000 [==============================] - 9s 154us/sample - loss: 2.7540 - acc: 0.8204 - val_loss: 2.7576 - val_acc: 0.8202
Epoch 2/3
roc-auc_train: 0.5 - roc-auc_val: 0.5                                                                                                     

Epoch 00002: saving model to weights-improvement-02-0.8202.hdf5
60000/60000 [==============================] - 8s 140us/sample - loss: 2.7540 - acc: 0.8204 - val_loss: 2.7576 - val_acc: 0.8202
Epoch 3/3
roc-auc_train: 0.5 - roc-auc_val: 0.5                                                                                                     

Epoch 00003: saving model to weights-improvement-03-0.8202.hdf5
60000/60000 [==============================] - 8s 

In [ ]:

# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs/fit

model-3

In [ ]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score
import datetime
class Metrics(tf.keras.callbacks.Callback):
  def __init__(self,validation_data=(X_test,Y_test),  interval=1):
        self.interval = interval
        self.X_val, self.y_val = validation_data
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []
  def on_epoch_end(self, epoch,logs={}):
        y_pred_train = self.model.predict(self.X_val)
        roc_train = roc_auc_score(self.y_val, y_pred_train)
        y_pred_val = self.model.predict(self.X_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        val_predict = (np.asarray(self.model.predict(self.X_val)))
        val_targ = self.y_val
        val_predict=np.argmax(val_predict, axis=1)
        val_targ=np.argmax(val_targ, axis=1)
        _val_f1 = f1_score(val_targ, val_predict, average= 'micro')
        _val_recall = recall_score(val_targ, val_predict, average= 'micro')
        _val_precision = precision_score(val_targ, val_predict, average= 'micro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("\rroc-auc_train: %s - roc-auc_val: %s " %(str(round(roc_train,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return
def scheduler(epoch,lr):
   if(epoch%3==0):
     lr *= 0.95
   return lr




m3= keras.models.Sequential()
m3.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(),input_shape=(784,)))
m3.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform()))
m3.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform()))
m3.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform()))
m3.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform()))
m3.add(tf.keras.layers.Dense(units =10,activation='softmax',kernel_initializer=tf.keras.initializers.he_uniform()))



history_own = Metrics()
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0, patience=1, min_lr=0.1)
terminate = tf.keras.callbacks.TerminateOnNaN()
callback12 = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=2)
filepath="weights-improvement-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',  verbose=1,save_weights_only=False, save_best_only=False , mode='max')
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback3 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,write_grads=True)


m3.compile(optimizer=tf.keras.optimizers.SGD(0.1),loss='binary_crossentropy',metrics=['accuracy'])
m3.fit(X_train,Y_train,epochs=3,validation_data=(X_test,Y_test),batch_size=16,callbacks=[history_own,reduce_lr,terminate,callback12,checkpoint,callback,tensorboard_callback3])


Train on 60000 samples, validate on 10000 samples
Epoch 1/3
roc-auc_train: 0.5 - roc-auc_val: 0.5                                                                                                     

Epoch 00001: saving model to weights-improvement-01-0.8196.hdf5
60000/60000 [==============================] - 9s 147us/sample - loss: 2.7586 - acc: 0.8201 - val_loss: 2.7668 - val_acc: 0.8196
Epoch 2/3
roc-auc_train: 0.5 - roc-auc_val: 0.5                                                                                                     

Epoch 00002: saving model to weights-improvement-02-0.8196.hdf5
60000/60000 [==============================] - 8s 139us/sample - loss: 2.7646 - acc: 0.8197 - val_loss: 2.7668 - val_acc: 0.8196
Epoch 3/3
roc-auc_train: 0.5 - roc-auc_val: 0.5                                                                                                     

Epoch 00003: saving model to weights-improvement-03-0.8196.hdf5
60000/60000 [==============================] - 8s 

model-4:

optimizer:adam,initializer:glorot_uniform()

In [ ]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score
import datetime
class Metrics(tf.keras.callbacks.Callback):
  def __init__(self,validation_data=(X_test,Y_test),  interval=1):
        self.interval = interval
        self.X_val, self.y_val = validation_data
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []
  def on_epoch_end(self, epoch,logs={}):
        y_pred_train = self.model.predict(self.X_val)
        roc_train = roc_auc_score(self.y_val, y_pred_train)
        y_pred_val = self.model.predict(self.X_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        val_predict = (np.asarray(self.model.predict(self.X_val)))
        val_targ = self.y_val
        val_predict=np.argmax(val_predict, axis=1)
        val_targ=np.argmax(val_targ, axis=1)
        _val_f1 = f1_score(val_targ, val_predict, average= 'micro')
        _val_recall = recall_score(val_targ, val_predict, average= 'micro')
        _val_precision = precision_score(val_targ, val_predict, average= 'micro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("\rroc-auc_train: %s -f1-score: %s" %(str(round(roc_train,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return
def scheduler(epoch,lr):
   if(epoch%3==0):
     lr *= 0.95
   return lr




m4= keras.models.Sequential()
m4.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.glorot_uniform(),input_shape=(784,)))
m4.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.glorot_uniform()))
m4.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.glorot_uniform()))
m4.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.glorot_uniform()))
m4.add(tf.keras.layers.Dense(units =50,activation='relu',kernel_initializer=tf.keras.initializers.glorot_uniform()))
m4.add(tf.keras.layers.Dense(units =10,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_uniform()))



history_own = Metrics()
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0, patience=1, min_lr=0.1)
terminate = tf.keras.callbacks.TerminateOnNaN()
callback12 = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=2)
filepath="weights-improvement-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc',  verbose=1,save_weights_only=False, save_best_only=False , mode='max')
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback4= tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,write_grads=True)


m4.compile(optimizer=tf.keras.optimizers.Adam(0.1),loss='binary_crossentropy',metrics=['accuracy'])
m4.fit(X_train,Y_train,epochs=3,validation_data=(X_test,Y_test),batch_size=16,callbacks=[history_own,reduce_lr,terminate,callback12,checkpoint,callback,tensorboard_callback4])


Train on 60000 samples, validate on 10000 samples
Epoch 1/3
roc-auc_train: 0.5 -f1-score: 0.5                                                                                                    

Epoch 00001: saving model to weights-improvement-01-0.8192.hdf5
60000/60000 [==============================] - 12s 195us/sample - loss: 2.7650 - acc: 0.8197 - val_loss: 2.7736 - val_acc: 0.8192
Epoch 2/3
roc-auc_train: 0.5 -f1-score: 0.5                                                                                                    

Epoch 00002: saving model to weights-improvement-02-0.8192.hdf5
60000/60000 [==============================] - 11s 184us/sample - loss: 2.7649 - acc: 0.8197 - val_loss: 2.7736 - val_acc: 0.8192
Epoch 3/3
roc-auc_train: 0.5 -f1-score: 0.5                                                                                                    

Epoch 00003: saving model to weights-improvement-03-0.8192.hdf5
60000/60000 [==============================] - 11s 183us/sample